In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 15 09:35:01 2020

@author: zxw
"""
# 引入库
import re
import pandas as pd
import time
import urllib.request
from lxml.html import fromstring
from bs4 import BeautifulSoup

# 下载链接
def download(url):
    print('Downloading:', url)
    request = urllib.request.Request(url)
    request.add_header('User-agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36') #进行伪装
    resp = urllib.request.urlopen(request)
    html = resp.read().decode('utf-8')
    return html

# 待爬取内容
name = []
year = []
rate = []
director = []
scriptwriter = []
protagonist = []
genre = []
country = []
language = []
length = []

# 循环爬取每页内容
for k in range(10):
    url = download('https://movie.douban.com/top250?start={}&filter='.format(k*25))
    time.sleep(3)   #间隔5s，防止被封禁
    #找出该页所有电影链接
    Links = re.findall('https://movie\.douban\.com/subject/[0-9]+/', url)
    movie_list = sorted(set(Links),key=Links.index)
    for movie in movie_list:
        url = download(movie)
        time.sleep(3)
        tree = fromstring(url)
        soup = BeautifulSoup(url)
        #利用正则表达式定位爬取
        name.append(re.search('(?<=(<span property="v:itemreviewed">)).*(?=(</span>))',url).group())
        year.append(re.search('(?<=(<span class="year">\()).*(?=(\)</span>))', url).group())
        rate.append(re.search('(?<=(<strong class="ll rating_num" property="v:average">)).*(?=(</strong>))', url).group())
        #利用xpath定位爬取
        director.append(tree.xpath('//*[@id="info"]/span[1]')[0].text_content())
        scriptwriter.append(tree.xpath('//*[@id="info"]/span[2]')[0].text_content())
        protagonist.append(tree.xpath('//*[@id="info"]/span[3]')[0].text_content())
        #利用find_all爬取
        genres = soup.find_all('span',{'property':'v:genre'})
        #将类型用'/'拼接
        temp = []
        for each in genres:
            temp.append(each.get_text())
        genre.append('/'.join(temp))
        #利用find定位爬取
        country.append(soup.find(text='制片国家/地区:').parent.next_sibling) #兄弟节点
        language.append(soup.find(text='语言:').parent.next_sibling)       
        length.append(soup.find('span',{'property':'v:runtime'}).get_text())

# 将list转化为dataframe
name_pd = pd.DataFrame(name)
year_pd = pd.DataFrame(year)
rate_pd = pd.DataFrame(rate)
director_pd = pd.DataFrame(director)
scriptwriter_pd = pd.DataFrame(scriptwriter)
protagonist_pd = pd.DataFrame(protagonist)
genre_pd = pd.DataFrame(genre)
country_pd = pd.DataFrame(country)
language_pd = pd.DataFrame(language)
length_pd = pd.DataFrame(length)
# 拼接
movie_data = pd.concat([name_pd,year_pd,rate_pd,director_pd,scriptwriter_pd,protagonist_pd,genre_pd,country_pd,language_pd,length_pd],axis=1)
movie_data.columns=['电影','年份','评分','导演','编剧','主演','类型','国家/地区','语言','时长']

#保留电影中文名
f = lambda x: re.split(' ',x)[0]
movie_data['电影'] = movie_data['电影'].apply(f)
#删去冗余部分
g = lambda x: x[4:-1] + x[-1]
movie_data['导演'] = movie_data['导演'].apply(g)
movie_data['编剧'] = movie_data['编剧'].apply(g)
movie_data['主演'] = movie_data['主演'].apply(g)
movie_data.head()

Downloading: https://movie.douban.com/top250?start=0&filter=
Downloading: https://movie.douban.com/subject/1292052/
Downloading: https://movie.douban.com/subject/1291546/
Downloading: https://movie.douban.com/subject/1292720/
Downloading: https://movie.douban.com/subject/1295644/
Downloading: https://movie.douban.com/subject/1292722/
Downloading: https://movie.douban.com/subject/1292063/
Downloading: https://movie.douban.com/subject/1291561/
Downloading: https://movie.douban.com/subject/1295124/
Downloading: https://movie.douban.com/subject/3541415/
Downloading: https://movie.douban.com/subject/3011091/
Downloading: https://movie.douban.com/subject/1889243/
Downloading: https://movie.douban.com/subject/1292064/
Downloading: https://movie.douban.com/subject/1292001/
Downloading: https://movie.douban.com/subject/3793023/
Downloading: https://movie.douban.com/subject/2131459/
Downloading: https://movie.douban.com/subject/1291549/
Downloading: https://movie.douban.com/subject/1292213/
Down

Downloading: https://movie.douban.com/subject/1294240/
Downloading: https://movie.douban.com/subject/25986180/
Downloading: https://movie.douban.com/subject/26325320/
Downloading: https://movie.douban.com/subject/1307315/
Downloading: https://movie.douban.com/subject/6307447/
Downloading: https://movie.douban.com/subject/1295399/
Downloading: https://movie.douban.com/subject/26628357/
Downloading: https://movie.douban.com/subject/1303037/
Downloading: https://movie.douban.com/top250?start=150&filter=
Downloading: https://movie.douban.com/subject/1417598/
Downloading: https://movie.douban.com/subject/1291585/
Downloading: https://movie.douban.com/subject/1306861/
Downloading: https://movie.douban.com/subject/4920389/
Downloading: https://movie.douban.com/subject/1297574/
Downloading: https://movie.douban.com/subject/1292208/
Downloading: https://movie.douban.com/subject/1578507/
Downloading: https://movie.douban.com/subject/1907966/
Downloading: https://movie.douban.com/subject/1395091/

,电影,年份,评分,导演,编剧,主演,类型,国家/地区,语言,时长
0,肖申克的救赎,1994,9.7,弗兰克·德拉邦特,弗兰克·德拉邦特 / 斯蒂芬·金,蒂姆·罗宾斯 / 摩根·弗里曼 / 鲍勃·冈顿 / 威廉姆·赛德勒 / 克兰西·布朗 / 吉...,剧情/犯罪,美国,英语,142分钟
1,霸王别姬,1993,9.6,陈凯歌,芦苇 / 李碧华,张国荣 / 张丰毅 / 巩俐 / 葛优 / 英达 / 蒋雯丽 / 吴大维 / 吕齐 / 雷汉...,剧情/爱情/同性,中国大陆 / 中国香港,汉语普通话,171分钟
2,阿甘正传,1994,9.5,罗伯特·泽米吉斯,艾瑞克·罗斯 / 温斯顿·格鲁姆,汤姆·汉克斯 / 罗宾·怀特 / 加里·西尼斯 / 麦凯尔泰·威廉逊 / 莎莉·菲尔德 / ...,剧情/爱情,美国,英语,142分钟
3,这个杀手不太冷,1994,9.4,吕克·贝松,吕克·贝松,让·雷诺 / 娜塔莉·波特曼 / 加里·奥德曼 / 丹尼·爱罗 / 彼得·阿佩尔 / 迈克尔...,剧情/动作/犯罪,法国 / 美国,英语 / 意大利语 / 法语,110分钟(剧场版)
4,泰坦尼克号,1997,9.4,詹姆斯·卡梅隆,詹姆斯·卡梅隆,莱昂纳多·迪卡普里奥 / 凯特·温丝莱特 / 比利·赞恩 / 凯西·贝茨 / 弗兰西丝·费舍...,剧情/爱情/灾难,美国,英语 / 意大利语 / 德语 / 俄语,194分钟


In [2]:
# 输出
outputpath='F:/douban_movie_all.csv'
movie_data.to_csv(outputpath,sep=',',index=False,header=True,encoding='utf_8_sig')